Bu kodun hazırlanmasında unsloth dökümantasyonundan yararlanılmıştır.

In [1]:
!pip install unsloth

Büyük Dil Modellerini olduğu gibi eğitmek için uzun zaman, fazla kaynak ve büyük veriye ihtiyaç vardır. Bu sorunun önüne geçmek için bir takım kütüphaneler ve yöntemler kullanılır. Kullanılan kütüphanelerden birisi de unsloth kütüphanesidir. Bu kütüphane istenilen açık kaynaklı büyük dil modellerini sisteme çeker ve çeşitli yöntemler sayesinde yüksek GB gereken veriyi küçülterek eğitimin kolay gerçekleştirilmesini ve çıktıların istenilen yerlerde saklanmasını sağlar. Unsloth kütüphanesinden kısaca bahsettiğimize göre yöntemlere değinelim.

Bahsedilen ve kodta geçen yöntemlerden birisi de 4bit kuantizasyon tekniğidir. Model ağırlıkları genellikle 32 bit float ya da 16 bit float olarak saklanır. Büyük modellerde bu ağırlıklar sistemde çok fazla yer kaplar. 4 bit kuantizasyon yöntemi ile ağırlıklar normalden 4 ile 8 kat daha az yer kaplar. Ancak saklanan verinin kesinliğinin (precision) düşük 16 ve 32 bitliğe göre daha düşük olmasından dolayı sonuçlar teorik olarak biraz düşük çıkabilir. Ancak modern tekniklerle (örn. Quantization-Aware Training, GPTQ) kayıp minimuma indirilebilir.

## Quantization-Aware Training (QAT)

Modeli FP32 veya FP16 olarak eğitirken aynı zamanda ağırlıkların kuantize edilmiş hallerini de simüle eder. Eğitim sırasında ağırlıklar hem float hem de kuantize edilmiş halde bulunur. Bu yöntemin amacı kuantizasyon sonrası performans kaybını önceden tahmin etmek ve minimize etmektir. Forward pass sırasında model, kuantize edilmiş ağırlıkları kullanır. Backpropagation sırasında gradient’ler float ağırlıklar üzerinden hesaplanır. Böylece model, kuantize edilmiş ağırlıkların getirdiği hataya "alışır" ve performans kaybı minimize edilir. Eğitim süresi artar, çünkü kuantizasyon simülasyonu ek hesaplama gerektirir.


## GPTQ (Gradient-based Post-Training Quantization)

Büyük modelleri eğittikten sonra kuantize eder. Gradient kullanarak 4-bit veya 3-bit kuantizasyon sırasında hatayı minimize eder. Model eğitildikten sonra ağırlıklar alınır. GPTQ, ağırlıkları grup veya blok bazlı olarak kuantize eder.
Kuantizasyon hatasını minimize etmek için local gradient’ler kullanılır (gradient descent ile optimize edilir, ama model yeniden eğitilmez). Sonuç olarak 4-bit veya 3-bit ağırlıklar, orijinal model performansına çok yakın bir doğrulukla elde edilir. Modeli yeniden eğitmek gerekmez. Post-training adımı, özellikle büyük modellerde zaman alabilir.

In [ ]:
from huggingface_hub import login
hf_token = 'YOUR_HF_TOKEN'
login(hf_token)

In [3]:
from unsloth import FastLanguageModel

max_seq_length = 2048 
dtype = None 
load_in_4bit = True


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-Distill-Llama-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = hf_token, 
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-11-07 12:13:53.923531: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762517633.949057     780 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762517633.957592     780 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.1: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [4]:
prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context. 
Write a response that appropriately completes the request. 
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning. 
Please answer the following medical question. 

### Question:
{}

### Response:
<think>{}"""

In [ ]:
# Model performansını önden test etmek için inference moduyla bir test
question = "A 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings, what would cystometry most likely reveal about her residual volume and detrusor contractions?"


FastLanguageModel.for_inference(model) 
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])


<think>
Okay, so I'm trying to figure out what the cystometry would show for this 61-year-old woman. Let me start by breaking down the information given.

First, the patient has a history of involuntary urine loss, specifically when she coughs or sneezes. That makes me think of stress urinary incontinence, which usually involves the urethra and is often due to weakened pelvic floor muscles or urethral issues. But she doesn't leak at night, which is interesting because that suggests it's not just a problem with bladder capacity, which is usually more of a concern for nighttime leaks.

She undergoes a gynecological exam and a Q-tip test. The Q-tip test is used to assess urethral function. If the test shows a positive result, it means there's a urethral obstruction, possibly due to a urethral incision or a urethral fold. If it's negative, it suggests the obstruction is at the bladder neck or the urethral orifice.

So, putting this together, if the Q-tip test is positive, that points towa

In [6]:
train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context. 
Write a response that appropriately completes the request. 
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning. 
Please answer the following medical question. 

### Question:
{}

### Response:
<think>
{}
</think>
{}"""

In [ ]:
EOS_TOKEN = tokenizer.eos_token  # EOS_TOKEN eklenmezse eğitim sonsuza kadar sürer.


def formatting_prompts_func(examples):
    inputs = examples["Question"]
    cots = examples["Complex_CoT"]
    outputs = examples["Response"]
    texts = []
    for input, cot, output in zip(inputs, cots, outputs):
        text = train_prompt_style.format(input, cot, output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,
    }

In [8]:
from datasets import load_dataset
dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT","en", split = "train[0:500]",trust_remote_code=True)
dataset = dataset.map(formatting_prompts_func, batched = True,)
dataset["text"][0]

`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'FreedomIntelligence/medical-o1-reasoning-SFT' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.
[datasets.load|ERROR]`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'FreedomIntelligence/medical-o1-reasoning-SFT' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.


"Below is an instruction that describes a task, paired with an input that provides further context. \nWrite a response that appropriately completes the request. \nBefore answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.\n\n### Instruction:\nYou are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning. \nPlease answer the following medical question. \n\n### Question:\nGiven the symptoms of sudden weakness in the left arm and leg, recent long-distance travel, and the presence of swollen and tender right lower leg, what specific cardiac abnormality is most likely to be found upon further evaluation that could explain these findings?\n\n### Response:\n<think>\nOkay, let's see what's going on here. We've got sudden weakness in the person's left arm and leg - and that screams something neuro-related, maybe a stroke?\n\nBut wait, there's more. The right lower leg i

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # Rank parametresidir. =! 0 olmalıdır. 8, 16, 32, 64, 128 değerleri önerilir.
    # Bu değer arttıkça model doğruluğu artar ancak overfitting riski de artar.
    # Eğitilecek katmanlar
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
     #lora parametreleri ve bias
    # özellikle lora da A B matrisinin katsayısı olarak lora_alpha / rank seçilir.
    lora_alpha=16,
    lora_dropout=0,  
    bias="none",  
    # Uzun contextlerde True ya da unsloth ifadesi kullanılır.
    use_gradient_checkpointing="unsloth",  
    random_state=3407,
    #rank stabilized lora aktif mi pasif mi olacak onu belirler.
    use_rslora=False,  
    loftq_config=None,
)

Unsloth 2025.11.1 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Modeli eğitmeden önce bütün modeli eğitmek fazlaca yer ve zaman içereceğinden dolayı PEFT (Parameter Efficient Fine Tuning) tekniğini kullanırız. Bu bize belli katmanlar üzerinde model eğitimini sağlar ve bütün modeli eğitmektense modelin başlı kısımlarını eğitiriz. Ayrıca PEFT ile günümüzde sıkça kullanılan ve PEFT'in alt yöntemlerinden biri olan LORA (Low Rank Adaptation) tekniğini de sıkça kullanırız.

### PEFT’in alt yöntemleri:

 PEFT bir çatı kavramdır. Altında farklı teknikler vardır:

###
Teknik	        Açıklama

LoRA:	        En popüler yöntem. Ağırlıklara düşük-rank matris ekler.

Prefix Tuning:	Girdi dizisine “özel tokenlar” ekleyerek ayar yapar.

Adapter Tuning:	Her katmana küçük ek sinir ağı modülleri ekler.

Prompt Tuning:	Modeli yeniden eğitmeden, giriş metnine sanal promptlar ekler.
###

Bugün en çok kullanılanı LoRA’dır.

### LoRA (Low-Rank Adaptation)

LoRA, büyük matrisleri tamamen güncellemek yerine, onlara küçük, düşük-rank eklemeler yapar. Bir model katmanında normalde şöyle bir ağırlık vardır:

𝑦 = 𝑊 * 𝑥

Burada 𝑊 genelde çok büyük (örneğin 4096×4096 boyutunda) bir matristir. LoRA bunu şu şekilde değiştirir:

𝑦 =(𝑊+𝐴𝐵)𝑥

Burada 𝐴 ve 𝐵 küçük, düşük-rank matrislerdir. 𝑟 bu matrislerin boyutunu belirleyen parametre (örneğin 4, 8, 16). Eğitim sırasında 𝑊 sabit kalır (freeze). Sadece 𝐴 ve 𝐵 öğrenilir. Bu sayede parametre sayısı 100 kat azalabilir. Performans çoğu zaman aynı kalır veya çok az düşer. Bu sayede eğitilecek parametre sayısı azalır, bellekte tasarruf sağlanır ve eğitim hızlanır.

![image.png](attachment:c07ed10e-db72-4371-9bef-f25967c697c6.png)

### PEFT + LoRA birlikte nasıl çalışır?

PEFT, “hangi parametreleri eğitelim?” sorusuna genel cevaptır. LoRA, bu stratejilerden biridir (en verimlisi). PEFT önceden eğitilmiş katmanları alır ve belirli modüllere LoRA ekler. LoRA ise bu katmanları eğitir. 

In [ ]:
from trl import SFTTrainer, SFTConfig

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # 4 adımda gradyanlar toplansın
        warmup_steps = 5,
        # num_train_epochs = 1, # Full training için 1 ata
        #Modeli test ettiğimiz için max adım sayısı belirliyoruz.
        max_steps = 60,
        learning_rate = 2e-4, # Uzun traininglerde 2e-5 e düşür
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Wandb, tensorboard vs kullanmak istemiyorsak none
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=8):   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 500 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.148500
2,2.151700
3,2.196500
4,2.000500
5,2.174400
6,1.964400
7,1.835700
8,1.814800
9,1.575000
10,1.508700


In [ ]:
question = "A 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings, what would cystometry most likely reveal about her residual volume and detrusor contractions?"


FastLanguageModel.for_inference(model)  
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])



<think>
Okay, let's see. She's a 61-year-old woman with a history of losing urine involuntarily during things like coughing or sneezing but not at night. That's pretty typical for someone with stress urinary incontinence. Stress incontinence usually happens when the pelvic floor muscles aren't strong enough to hold the bladder closed during activities.

Now, when they did the Q-tip test, I'm guessing it came back positive. The Q-tip test is a quick way to check for urethral obstruction. If the test is positive, that means there's some obstruction in the urethra, and it's probably affecting how she urinates.

So, the fact that her Q-tip test came back positive suggests there's some urethral obstruction. This could mean that her urethra isn't opening up smoothly, which might be causing some difficulty with urination. It might also be contributing to her stress incontinence.

Now, thinking about cystometry, what does it tell us? Cystometry helps us see how her bladder behaves under diffe

In [13]:
new_model_local = "DeepSeek-R1-Medical-COT"
model.save_pretrained(new_model_local) 
tokenizer.save_pretrained(new_model_local)

model.save_pretrained_merged(new_model_local, tokenizer, save_method = "merged_16bit",)

config.json:   0%|          | 0.00/891 [00:00<?, ?B/s]

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Checking cache directory for required files...
Cache check failed: model-00001-of-00004.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Unsloth: Preparing safetensor model files: 100%|██████████| 4/4 [01:15<00:00, 18.84s/it]


Note: tokenizer.model not found (this is OK for non-SentencePiece models)


Unsloth: Merging weights into 16bit: 100%|██████████| 4/4 [02:34<00:00, 38.56s/it]


Unsloth: Merge process complete. Saved to `/kaggle/working/DeepSeek-R1-Medical-COT`
